# YAKE

In [ ]:
# !pip install yake

In [ ]:
import yake

def extract_keywords_yake(text, max_keywords=5):
    # YAKE 키워드 추출기 생성
    kw_extractor = yake.KeywordExtractor()
    # 텍스트에서 키워드 추출
    keywords = kw_extractor.extract_keywords(text)
    # 상위 max_keywords 개의 키워드 반환
    return [kw[0] for kw in keywords[:max_keywords]]

# 예제 텍스트
text = "KeyBERT is a minimal and easy-to-use keyword extraction technique."

# YAKE를 사용하여 키워드 추출
keywords = extract_keywords_yake(text)
print("Extracted Keywords:", keywords)

# 검증
expected_keywords = ["KeyBERT", "keyword", "extraction", "technique", "easy-to-use"]
print("Expected Keywords:", expected_keywords)

# 간단한 검증
correct_keywords = set(keywords) & set(expected_keywords)
print("Correct Keywords:", correct_keywords)


# TF-IDF

In [ ]:
# sudo apt install default-jdk: ubuntu에서 java를 찾을 수 없을 때
# !pip install konlpy

In [ ]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
okt = Okt()

In [ ]:
docs = ["파이썬에서 다중 라인 문자열의 사용 예는 무엇인가요"]
text = ' '.join(docs)

In [ ]:
# 명사 추출
nouns = okt.nouns(text)
processed_text = ' '.join(nouns)
print(processed_text)

# TF-IDF 벡터
vectorizer = TfidfVectorizer()

# 텍스트를 TF-IDF벡터로 변환
tfidf_matrix = vectorizer.fit_transform([processed_text])
print(tfidf_matrix)

# 단어와 그에 대한 TF-IDF값 얻기
feature_names = vectorizer.get_feature_names_out()
print(feature_names)
tfidf_values = tfidf_matrix.toarray().tolist()[0]
print(tfidf_values)

# 높은 TF-IDF값을 가진 단어 순서대로 정렬
keywords = [feature_names[i] for i in sorted(range(len(tfidf_values)), key=lambda k: tfidf_values[k], reverse=True)]
print(keywords)

# keybert + kiwi
- https://datainclude.me/posts/Keybert%EC%99%80_kiwi%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D%EA%B8%B0%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC_%ED%82%A4%EC%9B%8C%EB%93%9C%EC%B6%94%EC%B6%9C_%ED%95%98%EA%B8%B0/

In [ ]:
# !pip install keybert
# !pip install kiwipiepy

In [ ]:
import pandas as pd
from keybert import KeyBERT
from transformers import BertModel
from kiwipiepy import Kiwi

text = '데이터프레임에서 특정 열을 추출하려면 어떻게 해야 하나요'

model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)

kiwi = Kiwi()
kiwi.analyze(text)

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

nouns = noun_extractor(text)

text = ' '.join(nouns)

keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=1)

keywords

In [ ]:
import pandas as pd
from keybert import KeyBERT
from transformers import BertModel
from kiwipiepy import Kiwi

# 모델 선택
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
kiwi = Kiwi()


In [ ]:
text = 'range() 함수를 사용한 반복문의 예를 들어보세요.'

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

nouns = noun_extractor(text)

text = ' '.join(nouns)

keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)

keywords

In [ ]:
import pandas as pd
from keybert import KeyBERT
from transformers import BertModel
from kiwipiepy import Kiwi

# 모델 선택
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
kiwi = Kiwi()

# 데이터 불러오기
df = pd.read_excel('../fasttext/data/train_set.xlsx')

# 불용어 파일 불러오기
with open('stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
            results.append(token)
    return results

# 키워드 추출 함수
def extract_keywords_from_df(df, stopwords, kw_model):
    keywords_list = []

    for _, row in df.iterrows():
        if row['label'] == 'yes':
            noun = noun_extractor(row['question'])
            text_noun = ' '.join(noun)
            keywords = kw_model.extract_keywords(text_noun, keyphrase_ngram_range=(1, 1), stop_words=stopwords, top_n=20)
            keywords_list.append(', '.join([kw[0] for kw in keywords]))
        else:
            keywords_list.append('') 
    return keywords_list


# 키워드 추출
df['keyword'] = extract_keywords_from_df(df, stopwords, kw_model)

# 결과를 엑셀 파일로 저장
df.to_csv('keyword.csv', index=False)

print("키워드 추출 및 저장이 완료되었습니다.")


# 실행파일

In [1]:
import os

# TensorFlow 경고 메시지 억제
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0: all messages, 1: info messages, 2: warnings, 3: errors
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import pandas as pd
from keybert import KeyBERT
from transformers import BertModel
from kiwipiepy import Kiwi

# bert와 keybert 모델을 활용하여 문장에서 명사를 추출하는 함수와 키워드를 추출하는 함수가 있는 객체입니다.
class Extract_keywords:
    def __init__(self, model_name='skt/kobert-base-v1', stopwords=None, top_n=5):  # stopwords와 top_n 기본값 설정
        self.model_name = model_name
        self.stopwords = stopwords if stopwords else []
        self.top_n = top_n
        self.kiwi = Kiwi()
        self.kw_model = self.load_kw_extractor_model()

    # 키워드 추출 모델 불러오는 함수
    def load_kw_extractor_model(self):
        model = BertModel.from_pretrained(self.model_name)
        kw_model = KeyBERT(model)
        return kw_model
    
    # 명사 추출 메서드
    def noun_extractor(self, text):
        results = []
        result = self.kiwi.analyze(text)
        for token, pos, _, _ in result[0][0]:
            if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
                results.append(token)
        return results

    # 데이터 프레임에서 키워드를 추출하는 메서드
    def extract_keywords_from_dataframe(self, dataframe, extract_column_name, label_column_name, label_value):
        max_keywords = 0
        keywords_list = []
        for _, row in dataframe.iterrows():
            # 라벨이 0인 컬럼만 불러오기
            if row[label_column_name] == label_value:
                noun = self.noun_extractor(row[extract_column_name])
                text_noun = ' '.join(noun)
                keywords = self.kw_model.extract_keywords(text_noun, keyphrase_ngram_range=(1, 1), stop_words=self.stopwords, top_n=self.top_n)
                keyword_texts = [kw[0] for kw in keywords]
                max_keywords = max(max_keywords, len(keyword_texts))
                keywords_list.append(keyword_texts)
            else:
                keywords_list.append([]) 
        
        # 키워드를 데이터프레임에 저장
        for i in range(max_keywords):
            col_name = f'keyword_{i+1}'
            dataframe[col_name] = [keywords[i] if i < len(keywords) else '픂뽉쌭' for keywords in keywords_list]
        return dataframe
    
    # 문장에서 키워드를 추출하는 메서드
    def extract_keywords_from_text(self, text):
        noun = self.noun_extractor(text)
        text_noun = ' '.join(noun)
        keywords = self.kw_model.extract_keywords(text_noun, keyphrase_ngram_range=(1, 1), stop_words=self.stopwords, top_n=self.top_n)
        return [kw[0] for kw in keywords]


In [ ]:
import pandas as pd
from keybert import KeyBERT
from transformers import BertModel
from kiwipiepy import Kiwi

# bert와 keybert 모델을 활용하여 문장에서 명사를 추출하는 함수와 키워드를 추출하는 함수가 있는 객체입니다.
class Extract_keywords:
    def __init__(self, model_name='skt/kobert-base-v1', stopwords=None, top_n=5):  # stopwords와 top_n 기본값 설정
        self.model_name = model_name
        self.stopwords = stopwords if stopwords else []
        self.top_n = top_n
        self.kiwi = Kiwi()
        self.kw_model = self.load_kw_extractor_model()

    # 키워드 추출 모델 불러오는 함수
    def load_kw_extractor_model(self):
        model = BertModel.from_pretrained(self.model_name)
        kw_model = KeyBERT(model)
        return kw_model
    
    # 명사 추출 메서드
    def noun_extractor(self, text):
        results = []
        result = self.kiwi.analyze(text)
        for token, pos, _, _ in result[0][0]:
            if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
                results.append(token)
        return results

    # 데이터 프레임에서 키워드를 추출하는 메서드
    def extract_keywords_from_dataframe(self, dataframe, extract_column_name, label_column_name, label_value):
        max_keywords = 0
        keywords_list = []
        for _, row in dataframe.iterrows():
            # 라벨이 0인 컬럼만 불러오기
            if row[label_column_name] == label_value:
                noun = self.noun_extractor(row[extract_column_name])
                text_noun = ' '.join(noun)
                keywords = self.kw_model.extract_keywords(text_noun, keyphrase_ngram_range=(1, 1), stop_words=stopwords, top_n=self.top_n)
                keyword_texts = [kw[0] for kw in keywords]
                max_keywords = max(max_keywords, len(keyword_texts))
                keywords_list.append(keyword_texts)
            else:
                keywords_list.append([]) 
        
        # 키워드를 데이터프레임에 저장
        for i in range(max_keywords):
            col_name = f'keyword_{i+1}'
            dataframe[col_name] = [keywords[i] if i < len(keywords) else '픂뽉쌭' for keywords in keywords_list]
        return dataframe
    
    # 문장에서 키워드를 추출하는 메서드
    def extract_keywords_from_text(self, text):
        noun = self.noun_extractor(text)
        text_noun = ' '.join(noun)
        keywords = self.kw_model.extract_keywords(text_noun, keyphrase_ngram_range=(1, 1), stop_words=self.stopwords, top_n=self.top_n)
        return [kw[0] for kw in keywords]

In [ ]:
# 데이터 불러오기
df = pd.read_excel('../fasttext/data/train_set.xlsx')

# 불용어 파일 불러오기
with open('stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

# Class 인스턴스 생성 
extractor = Extract_keywords(stopwords=stopwords, top_n=5)

# 키워드 추출하여 데이터프레임 업데이트
df = extractor.extract_keywords_from_dataframe(df, extract_column_name='question', label_column_name='label', label_value='yes')

In [ ]:
# 확인
df.head()

In [ ]:
# csv파일로 내보내기
df.to_csv('keyword.csv')

In [ ]:
## 단일 텍스트
# 명사만 추출
text_noun = extractor.noun_extractor("판다스 데이터 프레임에대해 설명해주세요")
# 키워드 추출
text_kw = extractor.extract_keywords_from_text("판다스 데이터 프레임에대해 설명해주세요")

print(text_noun)
print(text_kw)

# Stopwords 만들기

In [ ]:
import numpy as np
import pandas as pd

# 엑셀 파일 불러오기
key_df = pd.read_excel('keyword.xlsx')

# 결측치 제거
key_df = key_df[key_df['keyword'].notna()]

# 'keyword' 열을 리스트로 변환
keywords = key_df['keyword'].to_list()

# 쉼표와 공백으로 구분된 각 키워드를 개별 단어로 분리
split_keywords = []
for keyword in keywords:
    split_keywords.extend([k.strip() for k in keyword.split(',')])

# 중복 제거 및 각 단어를 작은따옴표로 감싸기
unique_keywords = list(set(split_keywords))
quoted_keywords = [f"'{word}'" for word in unique_keywords]

# 리스트 형태로 표현
result = f"[{', '.join(quoted_keywords)}

print(result)


In [ ]:
if '데이터 프레임' in result:
    print('yes')
else:
    print('no')